In [1]:
from openai import OpenAI
import os

os.environ['TOGETHER_API_KEY'] = 'bf917c4afd545fdc63dbc3a62129880c51b441895bcbaf685dcf2792e77e9801'
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

client = OpenAI(
  api_key=TOGETHER_API_KEY,
  base_url='https://api.together.xyz/v1',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "system",
      "content": "You will get a list of words and you will have to find which word contains a sub-word of an animal.",
    },
    {
      "role": "user",
      "content": "Here is the list: storm, phone, million, field, sunrise.",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  temperature = 0
)

print(chat_completion.choices[0].message.content)

 The word "field" contains a sub-word of an animal, specifically "field" is a type of land where various animals may live and roam. However, if we are looking for a more specific animal name as a sub-word, then none of the words in the list contain a sub-word that is the name of an animal.


list of a tuple that contains: {category, word, sub-word}

In [10]:
from langchain import PromptTemplate, OpenAI

# Define a simple prompt template as a Python string

prompt_template = PromptTemplate.from_template("""
Identify the word from the given list that contains a subword of an {items_obj} within it?
The list: [{items_list}]
""")

items_list = ["storm", "specification", "field"]
items_obj = "animal"
prompt = prompt_template.format(items_obj = items_obj, items_list = items_list)
llm = OpenAI(
  api_key=TOGETHER_API_KEY,
  base_url='https://api.together.xyz/v1',
  model="mistralai/Mixtral-8x7B-Instruct-v0.1"
)
response = llm.predict(
    text=prompt
)

c:\Users\liore\miniconda3\envs\nlp_course\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [33]:
# Your list of strings
input_list = ["No", "Yes", "Yes", "No", "No", "Yes"]

# Split the list into "Yes" and "No" sublists
yes_list = [x for x in input_list if x == "Yes"]
no_list = [x for x in input_list if x == "No"]

# Initialize the result list
result_list = []

# Interleave the "Yes" and "No" sublists
for yes, no in zip(yes_list, no_list):
    result_list.extend([yes, no])

# Append any remaining elements from the longer sublist
if len(yes_list) > len(no_list):
    result_list.append(yes_list[-1])
elif len(no_list) > len(yes_list):
    result_list.append(no_list[-1])

print(result_list)


['Yes', 'No', 'Yes', 'No', 'Yes', 'No']


In [1]:
import gensim.downloader as dl
word2vec_model = dl.load("word2vec-google-news-300")

In [7]:
import nltk
from nltk.corpus import brown, stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from collections import Counter
import numpy as np 
import gensim.downloader as dl

# Download brown corpus
# nltk.download('brown')
# nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()

# Function to lemmatize words
def lemmatize_word(word, stop_words):
    if not all(char.isalpha() for char in word) or (word in stop_words):
        # Return the word unchanged if it contains digits
        return None
    
    return lemmatizer.lemmatize(word)

def read_categories_files():
    item_types = ['animal', 'body_part', 'vehicle_list', 'color', 'food', 'fruit', 'musical']
    types_dict = {}
    for item_type in item_types:
        with open(f'{item_type}_list.txt', 'r') as file:
            file_contents = [line.strip().lower() for line in file.readlines()]

        types_dict[item_type] = file_contents

    return types_dict

def get_subwords_per_item(word2vec_model, types_dict, corpus_base_word, words_freq):
    subwords_list_tuple = [] 
    word2_vec_vocab = word2vec_model.vocab
    for word in corpus_base_word:
        for item_category, items in types_dict.items():
            for sub_word in items:
                if (sub_word in word) and (not sub_word == word) and (word2_vec_vocab.get(sub_word) and word2_vec_vocab.get(word)) and\
                    (words_freq[sub_word] != 0 and words_freq[word] != 0):
                    subwords_list_tuple.append((word2vec_model.similarity(sub_word, word), item_category, sub_word, words_freq[sub_word], word, words_freq[word]))  
    
    return subwords_list_tuple

# Get all words from the Brown corpus
stop_words = set(stopwords.words('english'))
brown_words = brown.words()
brown_words = list(lemmatize_word(word.lower(), stop_words) for word in brown_words if lemmatize_word(word.lower(), stop_words) is not None)
# Lemmatize each word and keep only the base form
base_words = set(brown_words)
corpus_base_word = list(base_words)
corpus_word_freq = Counter(brown_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
corpus_word_freq_df = pd.DataFrame(corpus_word_freq.values(), corpus_word_freq.keys(), columns=["Word_freq"])
subwords_list_tuple = get_subwords_per_item(word2vec_model, read_categories_files(), corpus_base_word, corpus_word_freq)
selection_base_df = pd.DataFrame(subwords_list_tuple, columns=['Similarity', 'Category', 'Subword', 'Subword_freq', 'Word', 'Word_freq'])
selection_base_df = selection_base_df[selection_base_df['Similarity'] <= 0.2]
#categories_with_multiple_options = df['Category'].value_counts()[df['Category'].value_counts() > 1].index
#df_filtered = df[df['Category'].isin(categories_with_multiple_options)]
grouped = selection_base_df.groupby('Category')
random_questions = []
for category, group in grouped:
    # Randomly select 10 "Subword" entries weighted by "Subword_freq"
    categories_subwords_distinct = group.drop_duplicates(subset=['Subword'])
    max_size = 5 if len(categories_subwords_distinct) >= 5 else len(categories_subwords_distinct)
    selected_subwords = np.random.choice(categories_subwords_distinct['Subword'],
                                        size=max_size,
                                        replace=False,
                                        p = categories_subwords_distinct['Subword_freq'] / categories_subwords_distinct['Subword_freq'].sum()
                                    )
    # Iterate over each selected "Subword"
    for subword in selected_subwords:
        #other_categories = selection_base_df[selection_base_df['Category'] != category]
        my_categories = selection_base_df[selection_base_df['Category'] == category]
        other_categories = corpus_word_freq_df[~corpus_word_freq_df.index.isin(my_categories['Word'])]
        category_words = my_categories[my_categories['Subword'].isin([subword])]
        other_categories_weights = other_categories['Word_freq'] / other_categories['Word_freq'].sum()
        catergory_words_weights = category_words['Word_freq'] / category_words['Word_freq'].sum()
        # Randomly select 3 "Word" entries weighted by "Word_freq" from other categories
        selected_wrong_words = np.random.choice(other_categories.index, size=3, replace=False, p=other_categories_weights)
        # Randomly select 1 "Word" entries weighted by "Word_freq" from my category
        selected_correct_words = np.random.choice(category_words['Word'], size=1, replace=False, p=catergory_words_weights)
        # Create a DataFrame with the selected "Subword" and "Word" entries
        random_questions.append((category, subword, selected_correct_words[0], selected_wrong_words))

df_options = pd.DataFrame(random_questions, columns=["Category", "Subword", "Word", "Options"])

a


In [11]:
df_options

,Category,Subword,Word,Options
0,animal,frog,leapfrog,"[wagon, david, splice]"
1,animal,dog,dogma,"[turn, wise, basement]"
2,animal,fish,selfish,"[partially, devoted, field]"
3,animal,cat,allocate,"[available, old, piddington]"
4,animal,fox,foxhole,"[maximum, determined, engine]"
5,body_part,eye,keyed,"[level, needed, since]"
6,body_part,foot,football,"[long, economy, flew]"
7,body_part,head,flathead,"[provide, charles, cps]"
8,body_part,mouth,portsmouth,"[mercer, make, dialed]"
9,body_part,hair,chairman,"[letter, purpose, plane]"


In [6]:
def read_categories_files():
    item_types = ['animal', 'body_part', 'vehicle_list', 'color', 'food', 'fruit', 'musical']
    types_dict = {}
    for item_type in item_types:
        with open(f'{item_type}_list.txt', 'r') as file:
            file_contents = [line.strip().lower() for line in file.readlines()]

        types_dict[item_type] = file_contents

    return types_dict

def get_subwords_per_item(word2vec_model, types_dict, corpus_base_word, words_freq):
    subwords_list_tuple = [] 
    word2_vec_vocab = word2vec_model.vocab
    for word in corpus_base_word:
        for item_category, items in types_dict.items():
            for sub_word in items:
                if (sub_word in word) and (not sub_word == word) and word2_vec_vocab.get(sub_word) and word2_vec_vocab.get(word):
                    try:
                        subwords_list_tuple.append((word2vec_model.similarity(sub_word, word), item_category, sub_word, words_freq[sub_word], word, words_freq[word]))
                    except:
                        subwords_list_tuple.append((0, item_category, sub_word, words_freq[sub_word], word, words_freq[word]))    
    
    return subwords_list_tuple

subwords_list_tuple = get_subwords_per_item(word2vec_model, read_categories_files(), corpus_base_word, corpus_word_freq)

In [8]:
selection_base_df = pd.DataFrame(subwords_list_tuple, columns=['Similarity', 'Category', 'Subword', 'Subword_freq', 'Word', 'Word_freq'])
selection_base_df.to_csv('test_subwords.csv')

In [ ]:
from openai import OpenAI
import os

os.environ['TOGETHER_API_KEY'] = 'bf917c4afd545fdc63dbc3a62129880c51b441895bcbaf685dcf2792e77e9801'
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

client = OpenAI(
  api_key=TOGETHER_API_KEY,
  base_url='https://api.together.xyz/v1',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "system",
      "content": "You will get a list of words and you will have to find which word contains a sub-word of an animal.",
    },
    {
      "role": "user",
      "content": "Here is the list: storm, phone, million, field, sunrise.",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  temperature = 0
)

print(chat_completion.choices[0].message.content)

 The word that contains a sub-word of an animal is "field." The sub-word "field" is a type of animal that lives in groups and eats grass. An alternate answer is "phone" because it contains the sub-word "phone," which is theoutdoor enclosure for certain types of animals, particularly large carnivores such as lions or tigers. However, this is a more obscure usage, and the word "field" should be considered the more likely answer.


list of a tuple that contains: {category, word, sub-word}

In [ ]:
from langchain import PromptTemplate, OpenAI

# Define a simple prompt template as a Python string

prompt_template = PromptTemplate.from_template("""
Identify the word from the given list that contains a subword of an {items_obj} within it?
The list: [{items_list}]
""")

items_list = ["storm", "specification", "field"]
items_obj = "animal"
prompt = prompt_template.format(items_obj = items_obj, items_list = items_list)
llm = OpenAI(
  api_key=TOGETHER_API_KEY,
  base_url='https://api.together.xyz/v1',
  model="mistralai/Mixtral-8x7B-Instruct-v0.1"
)
response = llm.predict(
    text=prompt
)

c:\Users\liore\miniconda3\envs\nlp_course\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
# Your list of strings
input_list = ["No", "Yes", "Yes", "No", "No", "Yes"]

# Split the list into "Yes" and "No" sublists
yes_list = [x for x in input_list if x == "Yes"]
no_list = [x for x in input_list if x == "No"]

# Initialize the result list
result_list = []

# Interleave the "Yes" and "No" sublists
for yes, no in zip(yes_list, no_list):
    result_list.extend([yes, no])

# Append any remaining elements from the longer sublist
if len(yes_list) > len(no_list):
    result_list.append(yes_list[-1])
elif len(no_list) > len(yes_list):
    result_list.append(no_list[-1])

print(result_list)


['Yes', 'No', 'Yes', 'No', 'Yes', 'No']


In [ ]:
import nltk
from nltk.corpus import brown
from nltk.stem import WordNetLemmatizer
import pandas as pd
from collections import Counter
import numpy as np 

# Download brown corpus
# nltk.download('brown')
# nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Function to lemmatize words
def lemmatize_word(word):
    if not all(char.isalpha() for char in word):
        # Return the word unchanged if it contains digits
        return None
    
    return lemmatizer.lemmatize(word)

def read_categories_files():
    item_types = ['animal', 'body_part', 'vehicle_list', 'color', 'food', 'fruit', 'musical']
    types_dict = {}
    for item_type in item_types:
        with open(f'{item_type}_list.txt', 'r') as file:
            file_contents = [line.strip().lower() for line in file.readlines()]

        types_dict[item_type] = file_contents

    return types_dict

def get_subwords_per_item(types_dict, corpus_base_word, words_freq):
    subwords_list_tuple = [] 
    for word in corpus_base_word:
        for item_category, items in types_dict.items():
            for sub_word in items:
                if (sub_word in word) and (not sub_word == word):
                    subwords_list_tuple.append((item_category, sub_word, words_freq[sub_word], word, words_freq[word]))
    
    return subwords_list_tuple

# Get all words from the Brown corpus
brown_words = brown.words()
brown_words = list(lemmatize_word(word.lower()) for word in brown_words if lemmatize_word(word.lower()) is not None)
# Lemmatize each word and keep only the base form
base_words = set(brown_words)
corpus_base_word = list(base_words)
corpus_word_freq = Counter(brown_words)

In [ ]:
random_questions

[('animal', 'deer', 'deerskin', array(['it', 'of', 'the'], dtype=object)),
 ('animal',
  'cat',
  'unification',
  array(['material', 'action', 'so'], dtype=object)),
 ('animal',
  'human',
  'humanness',
  array(['see', 'will', 'their'], dtype=object)),
 ('animal', 'bat', 'battle', array(['scalp', 'quite', 'left'], dtype=object)),
 ('animal', 'rat', 'grateful', array(['the', 'stare', 'to'], dtype=object)),
 ('animal',
  'ant',
  'instant',
  array(['wa', 'hiding', 'thorough'], dtype=object)),
 ('animal',
  'dog',
  'dogmatically',
  array(['applied', 'a', 'of'], dtype=object)),
 ('animal',
  'cat',
  'application',
  array(['daily', 'Someone', 'encountered'], dtype=object)),
 ('animal', 'rat', 'accurate', array(['a', 'You', 'the'], dtype=object)),
 ('animal',
  'cat',
  'catalogue',
  array(['bulk', 'are', 'bright'], dtype=object)),
 ('body_part',
  'back',
  'background',
  array(['deemed', 'Alternately', 'age'], dtype=object)),
 ('body_part',
  'hand',
  'handing',
  array(['ballist

In [ ]:


subwords_list_tuple = get_sub_words_per_item(get_items(), corpus_base_word, corpus_word_freq)
df = pd.DataFrame(subwords_list_tuple, columns=['Category', 'Subword', 'Subword_freq', 'Word', 'Word_freq'])
print(df)

       Category Subword  Subword_freq                         Word  Word_freq
0        animal     rat             6                    generator         14
1     body_part     leg            53                 legitimately          2
2         fruit    date           102                       update          1
3     body_part    shin             3                 establishing         26
4     body_part    face           370            surface-declaring          1
...         ...     ...           ...                          ...        ...
7624       food    stew             5                  stewardship          1
7625      color     tan             7                    pakistani          0
7626     animal     cat            22                        cat's          1
7627  body_part    fist            25  mailed-fist-in-velvet-glove          1
7628     animal     rat             6                    ratcliffe          0

[7629 rows x 5 columns]


In [ ]:
selected_subwords

array(['piano', 'drum', 'drum', 'drum', 'drum', 'drum', 'drum', 'drum',
       'drum', 'flute'], dtype=object)

In [ ]:

base_words


{'agony',
 'impudence',
 'generator',
 'terrours',
 'jure',
 'puerile',
 'legitimately',
 'bombardment',
 'sh',
 'numeral',
 'update',
 'segur',
 'establishing',
 'convert',
 'patty',
 'cotton',
 'eve',
 'filagree',
 'truck',
 'yujobo',
 'cult',
 'mauve',
 'hammett',
 'state',
 'trimmer',
 'thwarted',
 'stonehenge',
 'brookfield',
 'recede',
 'defiantly',
 'cheerfully',
 'zipped',
 'partner',
 'investigator',
 'abbey',
 'harmonic',
 'distally',
 'doctorate',
 'responsiveness',
 'pegged',
 'romantically',
 'melon',
 'grandiloquent',
 'spot',
 'alsatian',
 'bergson',
 'srelease',
 'timeworn',
 'confession',
 'filmed',
 'given',
 'presently',
 'conspired',
 'nose',
 'prorate',
 'reproducibly',
 'vector',
 'disease',
 'library',
 'inquire',
 'allegro',
 'varlaam',
 'persianesque',
 'acknowledges',
 'imitating',
 'nuisance',
 'relayed',
 'byword',
 'happens',
 'abandoning',
 'palliative',
 'jameson',
 'aniseikonic',
 'soaring',
 'congregated',
 'extensive',
 'irreversibly',
 'chairing',
 'r

In [ ]:
random_questions

[('bat', 'battle', array(['police', 'scared', 'prepared'], dtype=object)),
 ('cat', 'indicate', array(['justice', 'important', 'care'], dtype=object)),
 ('human',
  'dehumanize',
  array(['earth', 'hypothalamus', 'year'], dtype=object)),
 ('horse',
  "horse's",
  array(['argument', 'bearing', 'goldsmith'], dtype=object)),
 ('cat',
  'educate',
  array(['yearly', 'impotent', 'diminishing'], dtype=object)),
 ('cat', 'vocation', array(['carpeting', 'early', 'backed'], dtype=object)),
 ('dog', 'dogma', array(['hear', 'boating', 'team'], dtype=object)),
 ('cat',
  'intensification',
  array(['learned', 'hundred', 'considered'], dtype=object)),
 ('rat',
  'democratic',
  array(['membership', 'recopied', 'fighting'], dtype=object)),
 ('human',
  'sub-human',
  array(['bloodstream', 'important', 'understanding'], dtype=object)),
 ('head', 'ahead', array(['important', 'separated', 'corner'], dtype=object)),
 ('face',
  'surfaced',
  array(['epiphyseal-diaphyseal', 'business', 'offered'], dtype=

In [ ]:
# add new categories - proffesion, countries, names
# add the similarities

subwords_list_tuple = sorted(subwords_list_tuple, key = lambda x: (x[2], x[-1]), reverse=True)
pd.DataFrame(subwords_list_tuple).to_csv("test_subwords.csv")

In [ ]:
#https://radimrehurek.com/gensim/corpora/wikicorpus.html
#https://dumps.wikimedia.org/enwiki/latest/
#https://github.com/janester/mad_libs/blob/master/List%20of%20Body%20Parts.txt body parts
#https://gist.github.com/borlaym/585e2e09dd6abd9b0d0a animals
#https://gist.githubusercontent.com/mordka/c65affdefccb7264efff77b836b5e717/raw/e65646a07849665b28a7ee641e5846a1a6a4a758/colors-list.txt colors
#https://gist.github.com/lasagnaphil/7667eaeddb6ed0c565f0cb653d756942 fruits
#https://github.com/imsky/wordlists/blob/master/nouns/food.txt food
#https://github.com/kennyn510/random-lists/blob/master/car-brands.txt vehicle
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus
from gensim import WikiCorpus

# path_to_wiki_dump = datapath("enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2")
path_to_wiki_dump = datapath("enwiki-latest-pages-articles.xml.bz2, or enwiki-YYYYMMDD-pages-articles.xml.bz2")

wiki_corpus = WikiCorpus(path_to_wiki_dump)

# Access data from the WikiCorpus
word_freq_counter = Counter()
cn = 0
# Iterate over documents in the corpus and count word frequencies
for text in wiki_corpus.get_texts():
    # Tokenize the text and update the word frequency counter
    word_freq_counter.update(text)
    cn += 1

# Get the most common words
most_common_words = word_freq_counter.most_common(20)  # Change 20 to the number of most common words you want

# Print the most common words
for word, freq in most_common_words:
    print(f"{word}: {freq}")
    
print(cn)

the: 34028
of: 18673
and: 14570
in: 12505
to: 10030
is: 5196
as: 4128
for: 3616
by: 3433
that: 3411
with: 3166
was: 3131
on: 2789
are: 2748
from: 2523
an: 2112
or: 2087
it: 1913
at: 1820
be: 1724
106


In [ ]:
from nltk.corpus import brown
import nltk
from collections import Counter

# nltk.download('brown')
len(brown.words())



1161192

In [ ]:
vocab_obj = model.wv.vocab["the"]
vocab_obj.count

C:\Users\liore\AppData\Local\Temp\ipykernel_4668\3478189546.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_obj = model.wv.vocab["deprecated"]


2747376

In [ ]:
w2c = dict()
for item in model.wv.vocab:
   w2c[item]=model.wv.vocab[item].count
w2cSorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=False))
w2cSortedList = list(w2cSorted.keys())

C:\Users\liore\AppData\Local\Temp\ipykernel_4668\3446493001.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  for item in model.wv.vocab:
C:\Users\liore\AppData\Local\Temp\ipykernel_4668\3446493001.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  w2c[item]=model.wv.vocab[item].count


In [ ]:
len(model.wv.vocab)

C:\Users\liore\AppData\Local\Temp\ipykernel_4668\4238153822.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  len(model.wv.vocab)


3000000

In [ ]:
import requests
import pandas as pd

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_animal_names"

# Fetch the HTML content
html_content = requests.get(url).content

# Read all tables from the HTML content
df_list = pd.read_html(html_content)

# Assuming the relevant table is the last one
animal_names_df = df_list[-1]

0        Aardvark
1       Albatross
2       Alligator
3      Alpaca[14]
4             Ant
          ...    
250    Wildebeest
251          Wolf
252     Wolverine
253        Wombat
255         Zebra
Name: A, Length: 234, dtype: object


In [36]:
random_questions

[('animal', 'deer', 'deerskin', array(['it', 'of', 'the'], dtype=object)),
 ('animal',
  'cat',
  'unification',
  array(['material', 'action', 'so'], dtype=object)),
 ('animal',
  'human',
  'humanness',
  array(['see', 'will', 'their'], dtype=object)),
 ('animal', 'bat', 'battle', array(['scalp', 'quite', 'left'], dtype=object)),
 ('animal', 'rat', 'grateful', array(['the', 'stare', 'to'], dtype=object)),
 ('animal',
  'ant',
  'instant',
  array(['wa', 'hiding', 'thorough'], dtype=object)),
 ('animal',
  'dog',
  'dogmatically',
  array(['applied', 'a', 'of'], dtype=object)),
 ('animal',
  'cat',
  'application',
  array(['daily', 'Someone', 'encountered'], dtype=object)),
 ('animal', 'rat', 'accurate', array(['a', 'You', 'the'], dtype=object)),
 ('animal',
  'cat',
  'catalogue',
  array(['bulk', 'are', 'bright'], dtype=object)),
 ('body_part',
  'back',
  'background',
  array(['deemed', 'Alternately', 'age'], dtype=object)),
 ('body_part',
  'hand',
  'handing',
  array(['ballist

In [22]:


subwords_list_tuple = get_sub_words_per_item(get_items(), corpus_base_word, corpus_word_freq)
df = pd.DataFrame(subwords_list_tuple, columns=['Category', 'Subword', 'Subword_freq', 'Word', 'Word_freq'])
print(df)

       Category Subword  Subword_freq                         Word  Word_freq
0        animal     rat             6                    generator         14
1     body_part     leg            53                 legitimately          2
2         fruit    date           102                       update          1
3     body_part    shin             3                 establishing         26
4     body_part    face           370            surface-declaring          1
...         ...     ...           ...                          ...        ...
7624       food    stew             5                  stewardship          1
7625      color     tan             7                    pakistani          0
7626     animal     cat            22                        cat's          1
7627  body_part    fist            25  mailed-fist-in-velvet-glove          1
7628     animal     rat             6                    ratcliffe          0

[7629 rows x 5 columns]


In [32]:
selected_subwords

array(['piano', 'drum', 'drum', 'drum', 'drum', 'drum', 'drum', 'drum',
       'drum', 'flute'], dtype=object)

In [40]:

base_words


{'agony',
 'impudence',
 'generator',
 'terrours',
 'jure',
 'puerile',
 'legitimately',
 'bombardment',
 'sh',
 'numeral',
 'update',
 'segur',
 'establishing',
 'convert',
 'patty',
 'cotton',
 'eve',
 'filagree',
 'truck',
 'yujobo',
 'cult',
 'mauve',
 'hammett',
 'state',
 'trimmer',
 'thwarted',
 'stonehenge',
 'brookfield',
 'recede',
 'defiantly',
 'cheerfully',
 'zipped',
 'partner',
 'investigator',
 'abbey',
 'harmonic',
 'distally',
 'doctorate',
 'responsiveness',
 'pegged',
 'romantically',
 'melon',
 'grandiloquent',
 'spot',
 'alsatian',
 'bergson',
 'srelease',
 'timeworn',
 'confession',
 'filmed',
 'given',
 'presently',
 'conspired',
 'nose',
 'prorate',
 'reproducibly',
 'vector',
 'disease',
 'library',
 'inquire',
 'allegro',
 'varlaam',
 'persianesque',
 'acknowledges',
 'imitating',
 'nuisance',
 'relayed',
 'byword',
 'happens',
 'abandoning',
 'palliative',
 'jameson',
 'aniseikonic',
 'soaring',
 'congregated',
 'extensive',
 'irreversibly',
 'chairing',
 'r

In [25]:
random_questions

[('bat', 'battle', array(['police', 'scared', 'prepared'], dtype=object)),
 ('cat', 'indicate', array(['justice', 'important', 'care'], dtype=object)),
 ('human',
  'dehumanize',
  array(['earth', 'hypothalamus', 'year'], dtype=object)),
 ('horse',
  "horse's",
  array(['argument', 'bearing', 'goldsmith'], dtype=object)),
 ('cat',
  'educate',
  array(['yearly', 'impotent', 'diminishing'], dtype=object)),
 ('cat', 'vocation', array(['carpeting', 'early', 'backed'], dtype=object)),
 ('dog', 'dogma', array(['hear', 'boating', 'team'], dtype=object)),
 ('cat',
  'intensification',
  array(['learned', 'hundred', 'considered'], dtype=object)),
 ('rat',
  'democratic',
  array(['membership', 'recopied', 'fighting'], dtype=object)),
 ('human',
  'sub-human',
  array(['bloodstream', 'important', 'understanding'], dtype=object)),
 ('head', 'ahead', array(['important', 'separated', 'corner'], dtype=object)),
 ('face',
  'surfaced',
  array(['epiphyseal-diaphyseal', 'business', 'offered'], dtype=

In [22]:
# add new categories - proffesion, countries, names
# add the similarities

subwords_list_tuple = sorted(subwords_list_tuple, key = lambda x: (x[2], x[-1]), reverse=True)
pd.DataFrame(subwords_list_tuple).to_csv("test_subwords.csv")

In [13]:
#https://radimrehurek.com/gensim/corpora/wikicorpus.html
#https://dumps.wikimedia.org/enwiki/latest/
#https://github.com/janester/mad_libs/blob/master/List%20of%20Body%20Parts.txt body parts
#https://gist.github.com/borlaym/585e2e09dd6abd9b0d0a animals
#https://gist.githubusercontent.com/mordka/c65affdefccb7264efff77b836b5e717/raw/e65646a07849665b28a7ee641e5846a1a6a4a758/colors-list.txt colors
#https://gist.github.com/lasagnaphil/7667eaeddb6ed0c565f0cb653d756942 fruits
#https://github.com/imsky/wordlists/blob/master/nouns/food.txt food
#https://github.com/kennyn510/random-lists/blob/master/car-brands.txt vehicle
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus
from gensim import WikiCorpus

# path_to_wiki_dump = datapath("enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2")
path_to_wiki_dump = datapath("enwiki-latest-pages-articles.xml.bz2, or enwiki-YYYYMMDD-pages-articles.xml.bz2")

wiki_corpus = WikiCorpus(path_to_wiki_dump)

# Access data from the WikiCorpus
word_freq_counter = Counter()
cn = 0
# Iterate over documents in the corpus and count word frequencies
for text in wiki_corpus.get_texts():
    # Tokenize the text and update the word frequency counter
    word_freq_counter.update(text)
    cn += 1

# Get the most common words
most_common_words = word_freq_counter.most_common(20)  # Change 20 to the number of most common words you want

# Print the most common words
for word, freq in most_common_words:
    print(f"{word}: {freq}")
    
print(cn)

the: 34028
of: 18673
and: 14570
in: 12505
to: 10030
is: 5196
as: 4128
for: 3616
by: 3433
that: 3411
with: 3166
was: 3131
on: 2789
are: 2748
from: 2523
an: 2112
or: 2087
it: 1913
at: 1820
be: 1724
106


In [42]:
from nltk.corpus import brown
import nltk
from collections import Counter

# nltk.download('brown')
len(brown.words())



1161192

In [28]:
vocab_obj = model.wv.vocab["the"]
vocab_obj.count

C:\Users\liore\AppData\Local\Temp\ipykernel_4668\3478189546.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_obj = model.wv.vocab["deprecated"]


2747376

In [29]:
w2c = dict()
for item in model.wv.vocab:
   w2c[item]=model.wv.vocab[item].count
w2cSorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=False))
w2cSortedList = list(w2cSorted.keys())

C:\Users\liore\AppData\Local\Temp\ipykernel_4668\3446493001.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  for item in model.wv.vocab:
C:\Users\liore\AppData\Local\Temp\ipykernel_4668\3446493001.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  w2c[item]=model.wv.vocab[item].count


In [35]:
len(model.wv.vocab)

C:\Users\liore\AppData\Local\Temp\ipykernel_4668\4238153822.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  len(model.wv.vocab)


3000000

In [27]:
import requests
import pandas as pd

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_animal_names"

# Fetch the HTML content
html_content = requests.get(url).content

# Read all tables from the HTML content
df_list = pd.read_html(html_content)

# Assuming the relevant table is the last one
animal_names_df = df_list[-1]

0        Aardvark
1       Albatross
2       Alligator
3      Alpaca[14]
4             Ant
          ...    
250    Wildebeest
251          Wolf
252     Wolverine
253        Wombat
255         Zebra
Name: A, Length: 234, dtype: object
